##############################################################

In [44]:
#   wget --timestamping https://tripod.nih.gov/ginas/gsrs/fullSeedData-2016-06-16.gsrs
#   zcat fullSeedData-2016-06-16.gsrs | cut -f3 > fullSeedData-2016-06-16.json
#    or
#    curl GINASURL | zcat | cut -f3 > fullSeedData-2016-06-16.json
#
#   Each row should be valid by itself, but all together they are not.
# 
#
#   resovoir_sample.awk -v K=10 fullSeedData-2016-06-16.json| \
#   awk 'BEGIN{print"{\"records\":["}{if(last)print last ",";last=$0}END{print last "]}"}'\
#   > fullSeedData-2016-06-16_sample.json

# wrap all the json objects in an array of records
# < fullSeedData-2016-06-16.json awk 'BEGIN{print"{\"records\":["}{if(last)print last ",";last=$0}END{print last "]}"}' > fullSeedData-2016-06-16_records.json

In [45]:
wc -l < ginas.nt

2474912


RDF ntriples have lines of the form "`<subject> <predicate> <object> .`"  
we should be able to confirm certain conditions hold:  
\>    The subject is always a Ginias identifier.  
\>    The predicate is now always a path to root (to be reinterperted as an ontological term).  
\>    The object is most variable but when it is a Ginas identifier. That identifier exists as a subject.  

In [46]:
grep -c "^<UNII:" ginas.nt
grep -c "^<GINASREF:" ginas.nt
grep -c "^<cite:" ginas.nt

1675640
754058
45214


The output is free of duplicate statements.  

What predicates are there and how many of each?

In [47]:
cut -f2 -d' ' ginas.nt| sort | uniq -c | sort -nr

 649506 <GINAS:reference_uuid>
 649506 <GINAS:reference_citation>
 303021 <OIO:hasdbxref>
  84979 <GINAS:uuid>
  84979 <GINAS:substanceClass>
  84977 <rdfs:label>
  70718 <GINAS:code_references>
  56543 <GINAS:structure_stereoCenters>
  56543 <GINAS:structure_opticalActivity>
  56543 <GINAS:structure_mwt>
  56543 <GINAS:structure_ezCenters>
  56543 <GINAS:structure_definedStereo>
  56543 <GINAS:structure_charge>
  56541 <GINAS:structure_formula>
  52852 <CHEBI:InChIKey>
  21935 <GINAS:ACTIVE_MOIETY>
  11158 <GINAS:PARENT_ORGANISM--PART_FRACTION>
  11083 <GINAS:PART_FRACTION--PARENT_ORGANISM>
   7881 <GINAS:PARENT--SALT_SOLVATE>
   7858 <GINAS:SALT_SOLVATE--PARENT>
   7437 <GINAS:mixture_component_substance>
   4933 <GINAS:SUBSTANCE--SUB_CONCEPT>
   4931 <GINAS:SUB_CONCEPT--SUBSTANCE>
   2442 <GINAS:PARENT--IMPURITY>
   2435 <GINAS:CONSTITUENT_ALWAYS_PRESENT--PARENT>
   2412 <GINAS:IMPURITY--PARENT>
   2227 <GINAS:PARENT--CONSTITUENT_ALWAYS_PRESENT>
   1925 <GINAS:mixture_component_type

the GINAS: predicates are a path within the JSON `record` object  

(except `ingredient` and `unii` which are technicaly sibblings of `record`)


Those need mapping to their proper ontology terms  

To be okay, unique object ids must not out number unique subject ids

In [48]:
cut -f1 -d' ' ginas.nt| grep "^<UNII:" |sort -u > ginas_subject.list
cut -f1 -d' ' ginas.nt| grep "^<GINASREF:" |sort -u > ginasref_subject.list
cut -f1 -d' ' ginas.nt| grep "^<cite:" |sort -u > ginascite_subject.list

In [49]:
wc -l < ginas_subject.list
wc -l < ginasref_subject.list
wc -l < ginascite_subject.list

80046
649506
4933


In [50]:
cut -f3 -d' ' ginas.nt | grep "^<UNII:" | sort -u > ginas_object.list
cut -f3 -d' ' ginas.nt | grep "^<GINASREF:" | sort -u > ginasref_object.list
cut -f3 -d' ' ginas.nt | grep "^<cite:" | sort -u > ginascite_object.list

In [51]:
wc -l < ginas_object.list
wc -l < ginasref_object.list
wc -l < ginascite_object.list

49192
649506
4931


Okay so far

In [52]:
comm -13 ginas_subject.list ginas_object.list

<UNII:C7P25NXB6G>
<UNII:GK21SBA74R>
<UNII:R33W215S92>


No output means all objects exist as subjects which is good.  

In [53]:
comm -13 ginasref_subject.list ginasref_object.list

We are set to construct some ugly CuriePrefixs and IRI

In [54]:
grep "> <GINAS:codes_code> <" ginas.nt | cut -f3- -d ' ' | cut -f1 -d \: | tr -d \< |  sort | uniq -c | sort -nr


In [55]:
jq '.[][].relationships[].type' fullSeedData-2016-06-16_records.json | sort | uniq -c | sort -nr

  21979 "ACTIVE MOIETY"
  11182 "PART/FRACTION->PARENT ORGANISM"
  11160 "PARENT ORGANISM->PART/FRACTION"
   8038 "SALT/SOLVATE->PARENT"
   7990 "PARENT->SALT/SOLVATE"
   4933 "SUBSTANCE->SUB_CONCEPT"
   4931 "SUB_CONCEPT->SUBSTANCE"
   3039 "IMPURITY->PARENT"
   2489 "CONSTITUENT ALWAYS PRESENT->PARENT"
   2472 "PARENT->IMPURITY"
   2461 "PARENT->CONSTITUENT ALWAYS PRESENT"
   1314 "ENANTIOMER->RACEMATE"
   1248 "RACEMATE->ENANTIOMER"
   1182 "METABOLITE->PARENT"
    945 "PARENT->METABOLITE"
    620 "BASIS OF STRENGTH"
    512 "TRANSPORTER->INHIBITOR"
    486 "INHIBITOR->TRANSPORTER"
    483 "IONIC MOIETY"
    313 "METABOLITE ACTIVE->PARENT"
    311 "UNSPECIFIED INGREDIENT"
    298 "METABOLITE INACTIVE->PARENT"
    267 "PARENT->METABOLITE ACTIVE"
    250 "PARENT->METABOLITE INACTIVE"
    193 "METABOLITE ACTIVE->PRODRUG"
    192 "PRODRUG->METABOLITE ACTIVE"
    172 "SUBSTRATE->METABOLIC ENZYME"
    169 "METABOLIC ENZYME->SUBSTRATE"
    157 "FATTY ACID->LIPID"
    143 "NON-LABELED->LABE

What are the CuriePrefixes in use?

In [56]:
cut -f1 -d' ' ginas.nt | cut -f1 -d\: | sort | uniq -c | sort -nr
echo ""
cut -f2 -d' ' ginas.nt | cut -f1 -d\: | sort | uniq -c | sort -nr

1675640 <UNII
 754058 <GINASREF
  45214 <cite

2034062 <GINAS
 303021 <OIO
  84977 <rdfs
  52852 <CHEBI


finding Object curie prefixs will be less realiable

In [57]:
cut -f3- -d' ' ginas.nt | grep "^<" | cut -f1 -d\: | sort | uniq -c | sort -nr | head -25

 720224 <GINASREF
 103454 <http
  91614 <UNII
  59232 <CAS
  20967 <ECHA (EC/EINECS)
  15680 <MERCK INDEX
  12786 <NCI_THESAURUS
  10870 <MESH
  10068 <RXCUI
   8874 <INN
   7750 <WIKIPEDIA
   5882 <NCBI TAXONOMY
   5475 <EVMPD
   5286 <NDF-RT
   4931 <cite
   4746 <WHO-VATC
   4569 <ITIS
   4525 <WHO-ATC
   3333 <GRIN
   2857 <EPA PESTICIDE CODE
   2563 <USDA PLANTS
   2330 <CFR
   1920 <DRUG BANK
   1902 <JECFA EVALUATION
   1354 <EMA ASSESSMENT REPORTS


Those are worth looking into but first, required resources: 

1) A source specific translation table that maps external strings into internal strings  
e.g. they say `NCBI TAXONOMY` we say `NCBITaxon`

2) A common mapping of short to long form prefixs or bases. That is, a file with the namespace or CURIE prefix mapped the the constant base portion of an IRI  
e.g.   `CAS	http://www.cas-no.org/`  

These first two are related in that every? element in the second column of the first mapping occurs as an element in the first column of the second mapping (a namepace).  
(I question _every_ b/c we may use this mechinism to normalize non ontoligical strings as well) 

3) A common mapping of human friendly labels to machine oriented terms per namespace  
e.g. `'RO:member_of' RO:0002350`

Where the first column is a string a programmer can reason about when they are creating or maintaining code. And the second is the CURIE for the ontological term being that will be included in the output. (the `namespace:label` will _not_ be emitted)  

some notes:  

i)  Including the namespace with the label is important because there are no constraints on what various ontologists lable things (including terms from different ontologies) so be selective when adopting a lable, favor the canonical source for a term.

ii) Labels may contain spaces and other identifier unfriendly aspects. Feedback pressure to keep them as sane as reasonable is a community service.

iii) Labels are apt to change more often than the underlying term, checking them before use is prudent.

--- 

Here we have enough information to structure and emit our data as Resource Discription Framework (RDF) triples anotated with ontological terms.


However to be a valid Web Ontology Language (OWL) representation the ontological terms need to be typed.

To my limited understanding, OWL can be represented a Directed Acyclic Graph (DAG) of RDF. Even if OWL itself is not constrained to be DAGs, usefully computable instances should not be more connected than a DAG.  All DAGs may be represented as a set of overlapping trees (hirearchies) with distinct roots but possible common internal and leaf nodes. 

When we annotate our data with an ontological term we also have to specify the root of the hirearchy we want the term to be considered part of incase it could be part of several. This matters when the OWL file is being reasoned over.  

There are severrial possible cases 
. the term is only within one hirearchy in which case the typing is moot (but still necessary). 
. The term is in multiple hirearchies but we only use it in one way in which case a common `term->type` mapping file is needed.

. The term is in multiple hirearchies and we use differently with different datasets in which case source specific `term->type` mapping file is needed.

. The term is in multiple hirearchies and we use differently within the same dataset in which case I might argue they should be different terms. prehaps by altering the namespace (but may muck up alot of nice consistency propertied we aught be checking).

made up example:

    WAG:whatever  WAG:abc123
    SWAG:whatever SWAG:abc123

where in the CuriePrefix_IRIPrefix mappinile both namespaceswould map the the sme base IRI

    WAG  http://example.org/
    SWAG http://example.org/

We could then globaly type the terms

    WAG:abc123  owl:turnip
    SWAG:abc123 owl:rutabegga

In all these cases we are able to isolate the OWL type metadata which will be used in concert with many other sources to extrapolate novle statements   
from the statments _this_ source explicitly asserts which is an excellent property when we want to establish consistancy and correctness of this source.


The least good option (and our current practice) is to hardcode the type with each term usage.


TODO: cleanup spellcheck

